<a href="https://colab.research.google.com/github/annika-fagerstrom/NumericalAnalysis/blob/main/Project_1_Rootfinding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1. Rootfinding

The <b>Gompertz curve</b> or Gompertz function is a type of mathematical model named after Benjamin Gompertz (1779-1865). It is a function that describes growth as being slowest at the start and end of a given time period. Population biology is especially concerned with the Gompertz function. This function is especially useful in describing the rapid growth of a certain population of organisms (such as tumors, bacteria, etc.) while also considering the eventual horizontal asymptote once the carrying capacity is determined. The function was originally designed to describe human mortality, but since has been modified to be applied in biology, with regards to detailing populations.

It is modeled as follows:

$$N(t) = N_0 \mathrm{exp}((\ln (N_I/N_0)) (1-\mathrm{exp}(-bt)) = N_0 e^{(\ln \frac{N_I}{N_0}) (1-e^{-bt})}$$

where $t$ is the time, $N(t)$ is the population at time $t$, $N_0$ is the initial population, $N_I$ is the plateau population number (the maximum capacity in the given situation), $b$ is the initial growth rate, and $exp(x)$ is the exponential function $e^x$. The unit for $N(t)$, $N_I$, and $N_0$ are millions, and the unit for $t$ is hours.

In this project, we are going to write computer programs that determine the amount of time that it takes for $N(t)$ to rise from the initial population $N_0 = 3\cdot 10^{-5}$ to $1$. We use $N_I = 10^3$ and $b = 0.12$. 

Note that the solution of $N(t) = 1$ is equivalent to $N(t) - 1 = 0$, so this is a root-finding problem.


#### 1. (20 pts) Create a Python function bisection(b) that finds the root of $N(t) - 1 = 0$ by the bisection method. The initial interval is $[0, b]$. 

<ul>
    <li>Use an error bound $10^{-6}$.</li>
    <li>Allow at most 1000 iterations.</li>
    <li>For each step, print the left endpoint $a_n$, the right endpoint $b_n$, and the approximation (= midpoint) $p_n$. </li>
</ul>

In [ ]:
import math # math for log and exponent functions
import pandas as pd # pandas to display the endpoints & midpoints nicely

#set our base parameters for the Gompertz function
N_zero = 3*(10**-5)
N_I = 10**3
b = 0.12

#define the Gompertz function
def N_of(t):
    exp1 = math.log(N_I/N_zero)
    exp2 = 1 - (math.e)**(-1*b*t)
    return N_zero*math.e**(exp1*exp2)

#define a slightly-adjusted function
def N_minus_1(t):
    return N_of(t) - 1

#this helper function builds a pandas dataframe from a list of lists
def bisection_df(point_list):
  df = pd.DataFrame(point_list, columns=["a_n", "b_n", "p_n"]) 
  df.index += 1 #make the index start at 1
  return df

def bisection(x):
  """
  Inputs: x (float or int)
  Outputs: A pandas dataframe w/ a_n, b_n, and p_n as columns
  
  This function takes an initial right endpoint x, and it performs the 
  bisection method on [0,x] using the function N(t) - 1.
  """
  #define some initial values
  error_bound = 10**(-6)
  a_n = 0
  b_n = x
  point_list = []
  i = 1
  f_a = N_minus_1(a_n) #evaluate function at a_n

  #perform the bisection method routine
  while i <= 1000: #stop the routine at 1000 iterations
    
    p_n = (1/2)*(a_n+b_n) #find the midpoint
    point_list.append([a_n,b_n,p_n]) #append the three points to a list of lists
    f_p = N_minus_1(p_n) #evaluate the function at the new midpoint

    #stop the routine if we reach the desired theoretical error bound
    if f_p==0 or (b_n - a_n)/(2) < error_bound:   
      return bisection_df(point_list)

    i += 1
    
    #otherwise, start the process over again with a new interval
    if (f_a*f_p)>0:
        a_n = p_n
        f_a = f_p
    else: 
        b_n = p_n

  #if the while loop ends, print an error message and return the dataframe
  print("Method did not reach the error bound after 1000 iterations")
  return bisection_df(point_list)

#test the function!
bisection(10)

,a_n,b_n,p_n
1,0.000000,10.000000,5.000000
2,5.000000,10.000000,7.500000
3,7.500000,10.000000,8.750000
4,7.500000,8.750000,8.125000
5,7.500000,8.125000,7.812500
6,7.500000,7.812500,7.656250
7,7.656250,7.812500,7.734375
8,7.656250,7.734375,7.695312
9,7.656250,7.695312,7.675781
10,7.656250,7.675781,7.666016


#### 2. (15 pts) Create a Python function newton(x) that finds the root of $N(t) - 1 = 0$ by Newton's method. The initial guess $p_0$ is $x$.

<ul>
    <li>Calculate the derivative $N'(t)$ manually and use it to code.</li>
    <li>Use an error bound $10^{-6}$. Note that the error size is estimated by $|p_{n+1} - p_n|$.</li>
    <li>Allow at most 1000 iterations.</li>
    <li>For each step, print $p_n$ and the estimation of the error $|p_n - p_{n-1}|$.</li>
</ul>

In [ ]:
#set our base parameters for the Gompertz function
N_zero = 3*(10**-5)
N_I = 10**3
b = 0.12

def newtons_df(point_list):
  df = pd.DataFrame(point_list, columns=["p_n", "current error"]) 
  df.index += 1 
  return df

def fnN_Prime (t):
    return N_zero*math.log(N_I/N_zero)*b*math.exp((math.log(N_I/N_zero))*(1-math.exp(-b*t)))*math.exp(-b*t)

def Newtons(x0):
  max_iter = 1000
  error_bound = 10**-6
  point_list = []
  p0 = x0
  n = 0 

  for n in range(0, max_iter):
    fp = N_minus_1(p0)
    Dfp = fnN_Prime(p0)
    p = p0 - fp/Dfp
    error = abs(p - p0)
    point_list.append([p, error])

    if Dfp == 0:
      print('Zero derivative. No solution found.')
      break

    elif abs(p - p0) < epsilon:
      print('Found solution after',n + 1,'iterations.')
      print('The root of the function is ',fp/Dfp)
      break

    p0 = p

  return newtons_df(point_list)

#Test the function 
Newtons(5)

Found solution after 10 iterations.
The root of the function is  4.24839822764736e-13


,p_n,current error
1,15.911383,1.091138e+01
2,12.706935,3.204449e+00
3,10.592574,2.114361e+00
4,9.098705,1.493869e+00
5,8.144989,9.537161e-01
6,7.732406,4.125829e-01
7,7.662897,6.950867e-02
8,7.661139,1.757718e-03
9,7.661138,1.095876e-06
10,7.661138,4.245493e-13


#### 3. (15 pts) Create a Python function secant(x0, x1) that finds the root of $N(t) - 1 = 0$ by secant method. $p_0 = x0$ and $p_1 = x1$. 

<ul>
    <li>Use an error bound $10^{-6}$. You may estimate the error size by $|p_{n} - p_{n-1}|$.</li>
    <li>Allow at most 1000 iterations.</li>
    <li>For each step, print $p_n$ and the estimation of an error $|p_n - p_{n-1}|$.</li>
</ul>

In [ ]:
#this helper function builds a pandas dataframe from a list of lists
def secant_df(point_list):
    df = pd.DataFrame(point_list, columns=["p_n", "error_size"]) 
    df.index += 1 #make the index start at 1
    return df

def secant(x0,x1):
    """
    Inputs: x0, x1 (each float or int)
    Outputs: A pandas dataframe w/ p_n and error size as columns
  
    This function takes two initial points x0 & x1, and then
    it performs the secant method using the function N(t) - 1.
    """
    #define some initial values
    point_list = []
    error_bound = 10**(-6)
    p0 = x0
    p1 = x1
    q0 = N_minus_1(p0)
    q1 = N_minus_1(p1)
    i = 2
    
    while i <= 1000:
        
        p = p1 - q1*(p1-p0)/(q1-q0) #find the zero of a tangent line
        error_size = abs(p - p1) #compute the error
        point_list.append([p,error_size]) #append our datapoints to a list of lists
        if error_size < error_bound:
            return secant_df(point_list)
        i +=1 
        
        #update p0, q0, p1, q1
        p0 = p1
        q0 = q1
        p1 = p
        q1 = N_minus_1(p)
    
    #if the while loop ends, print an error message and return the dataframe
    print("Method did not reach the error bound after 1000 iterations")
    return secant_df(point_list)
    
#test it!
secant(5,10)

,p_n,error_size
1,5.865478,4.134522e+00
2,6.505617,6.401388e-01
3,8.952481,2.446864e+00
4,7.177684,1.774796e+00
5,7.472724,2.950399e-01
6,7.697307,2.245825e-01
7,7.658662,3.864453e-02
8,7.661107,2.444338e-03
9,7.661138,3.159393e-05
10,7.661138,2.771497e-08
